# 2) Data Analysis

In [1]:
import pandas as pd
import random
import os
import openai
import json
import re
import matplotlib.pyplot as plt

In [2]:
#Load Dataset
Survey_1 = pd.read_csv("/work/data/2) cleaned_data/Cleaned - Survey 1.csv")
Survey_2 = pd.read_csv("/work/data/2) cleaned_data/Cleaned - Survey 2.csv")
Survey_3 = pd.read_csv("/work/data/2) cleaned_data/Cleaned - Survey 3.csv")

In [3]:
Response_Source = ["_D_", "_H_", "_L_"]
Medical_Domain = ["_Preventative_", "_Conditions_", "_Diagnostic", "_Procedures_", "_Medications_", "_Recovery_"]
Question_Names_Eval = ["Und_Q", "Und_R", "Valid", "Trust", "Satis", "Follow", "Action", "Info"]
Question_Type_Eval = ["_2_1", "_4_1", "_6", "_7_1", "_8_1", "_10_1", "_11_1", "_12_1"]
Question_Names_S2 = ["Und_Q", "Und_R", "AI_or_Human", "Confidence"]
Question_Type_S2 = ["_2_1", "_4_1", "_6", "_7_1"]

### Survey 1 - Data Analysis

Survey 1 - Dataset Extraction

In [4]:
S1_Lists = {"Und_Q": [], "Und_R": [], "Valid": [], "Trust": [], "Satis": [], "Follow": [], "Action": [], "Info": [], "_D_": [], "_H_": [], "_L_": [], "_Preventative_": [], "_Conditions_": [], "_Diagnostic": [], "_Procedures_": [], "_Medications_": [], "_Recovery_":[]}

Survey_1_df = pd.DataFrame(Survey_1)
column_headers = list(Survey_1_df)

for column in column_headers:
    for i in range(len(Question_Type_Eval)):
        question = Question_Type_Eval[i]
        ques_title = Question_Names_Eval[i]
        if question in column:
            S1_Lists[ques_title].append(column)       
        
    for source in Response_Source:
        if source in column:
            S1_Lists[source].append(column)
    
    for domain in Medical_Domain:
        if domain in column:
            S1_Lists[domain].append(column)


dataset_1 = pd.DataFrame()
dataset_1.insert(0, "Question Type", 0)
dataset_1.insert(1, "Response Scores", 0)
dataset_1.insert(2, "Response Source", 0)
dataset_1.insert(3, "Participant ID", 0)
dataset_1.insert(4, "Question ID", 0)

Survey 1 - Functions

In [5]:
def Likert(Question, Dataset, Stat_dataset, Survey_Str, Survey_Dict) :

    Source_Str = ["Doctor", "High Accuracy AI", "Low Accuracy AI"]

    for i in range(len(Response_Source)):
        Source = Response_Source[i]
        Source_list = Survey_Dict[Source]
        Ques_list = Survey_Dict[Question]

        Sample = sorted(list(set(Ques_list) & set(Source_list)))
     
        for col in Sample:
            Question_ID = col
            Temp_column = Dataset.loc[:, col]
            Participants = Dataset.loc[:, "prolific_id"]
        
            for j in range(len(Temp_column)):
                resp = Temp_column[j]
                Participant_ID = Participants[j]
                try:                
                    if 0 <= float(resp) <= 5:
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, float(resp), Source_Str[i], Participant_ID, Question_ID]
                except:
                    pass

    return('Done')



def yes_or_no(Question, Dataset, Stat_dataset, Survey_Dict) :

    Source_Str = ["Doctor", "High Accuracy AI", "Low Accuracy AI"]

    for i in range(len(Response_Source)):
        Source = Response_Source[i]
        Source_list = Survey_Dict[Source]
        Ques_list = Survey_Dict[Question]

        Sample = sorted(list(set(Ques_list) & set(Source_list)))

        for col in Sample:
            Question_ID = col
            Temp_column = Dataset.loc[:, col]
            Participants = Dataset.loc[:, "prolific_id"]
        
            for j in range(len(Temp_column)):
                Participant_ID = Participants[j]
                resp = Temp_column[j]
                try:                
                    if str(resp) == "Yes":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 1, Source_Str[i], Participant_ID, Question_ID]
                    if str(resp) == "No":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 0, Source_Str[i], Participant_ID, Question_ID]

                except:
                    pass

    return('Done')

Survey 1 - Execution

In [6]:
for ques in Question_Names_Eval:
    if ques != "Valid":
        Likert(ques, Survey_1, dataset_1, "Survey 1", S1_Lists)
    else:
        yes_or_no(ques, Survey_1, dataset_1, S1_Lists)

### Survey 2 - Data Analysis

Survey 2 - Dataset Extraction

In [7]:
S2_Lists = {"Und_Q": [], "Und_R": [], "AI_or_Human": [], "Confidence": [], "_D_": [], "_H_": [], "_L_": [], "_Preventative_": [], "_Conditions_": [], "_Diagnostic": [], "_Procedures_": [], "_Medications_": [], "_Recovery_":[]}

Survey_2_df = pd.DataFrame(Survey_2)
column_headers = list(Survey_2_df)

for column in column_headers:
    for i in range(len(Question_Type_S2)):
        question = Question_Type_S2[i]
        ques_title = Question_Names_S2[i]
        if question in column:
            S2_Lists[ques_title].append(column)       
        
    for source in Response_Source:
        if source in column:
            S2_Lists[source].append(column)
    
    for domain in Medical_Domain:
        if domain in column:
            S2_Lists[domain].append(column)


dataset_2_1 = pd.DataFrame()
dataset_2_1.insert(0, "Question Type", 0)
dataset_2_1.insert(1, "Response Scores", 0)
dataset_2_1.insert(2, "Response Source", 0)
dataset_2_1.insert(3, "Participant ID", 0)
dataset_2_1.insert(4, "Question ID", 0)

dataset_2_2 = pd.DataFrame()
dataset_2_2.insert(0, "Question Type", 0)
dataset_2_2.insert(1, "Incorrect/Correct", 0)
dataset_2_2.insert(2, "Response Scores", 0)
dataset_2_2.insert(3, "Confidence", 0)
dataset_2_2.insert(4, "Response Source", 0)
dataset_2_2.insert(5, "Participant ID", 0)
dataset_2_2.insert(6, "Question ID", 0)


Survey 2 - Functions

In [8]:
def AI_or_Human(Question, Dataset, Stat_dataset, Survey_Dict) :

    Source_Str = ["Doctor", "High Accuracy AI", "Low Accuracy AI"]

    for i in range(len(Response_Source)):
        Source = Response_Source[i]
        Source_list = Survey_Dict[Source]
        Ques_list = Survey_Dict[Question]

        Sample = sorted(list(set(Ques_list) & set(Source_list)))

        for col in Sample:
            Question_ID = col
            Temp_column = Dataset.loc[:, col]
            conf_str = str(col)
            conf = conf_str[0:-1] + '7_1'
            Conf_column = Dataset.loc[:, conf]
            Participants = Dataset.loc[:, "prolific_id"]

            for j in range(len(Temp_column)):
                resp = Temp_column[j]
                confidence = Conf_column[j]
                Participant_ID = Participants[j]

                try: 
                    if str(resp) == "Doctor":
                        if Source_Str[i] == "Doctor":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Correct", 1, confidence, Source_Str[i], Participant_ID, Question_ID]
                        if Source_Str[i] == "High Accuracy AI":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Incorrect", 0, confidence, Source_Str[i], Participant_ID, Question_ID]
                        if Source_Str[i] == "Low Accuracy AI":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Incorrect", 0, confidence, Source_Str[i], Participant_ID, Question_ID]

                    if str(resp) == "AI Text Generator":
                        if Source_Str[i] == "Doctor":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Incorrect", 0, confidence, Source_Str[i], Participant_ID, Question_ID]
                        if Source_Str[i] == "High Accuracy AI":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Correct", 1, confidence, Source_Str[i], Participant_ID, Question_ID]
                        if Source_Str[i] == "Low Accuracy AI":
                            Stat_dataset.loc[len(Stat_dataset)] = [Question, "Correct", 1, confidence, Source_Str[i], Participant_ID, Question_ID]

                except:
                    pass

    return('Done')


Survey 2 - Execution

In [9]:
Likert("Und_Q", Survey_2, dataset_2_1, "Survey 2", S2_Lists)
Likert("Und_R", Survey_2, dataset_2_1, "Survey 2", S2_Lists)
AI_or_Human("AI_or_Human", Survey_2, dataset_2_2, S2_Lists)

'Done'

### Survey 3 - Data Analysis

Survey 3 - Dataset Extraction

In [10]:
Random_Source = Survey_3.loc[:, "Assigned_Conditions"]

S3_Lists = {"Und_Q": [], "Und_R": [], "Valid": [], "Trust": [], "Satis": [], "Follow": [], "Action": [], "Info": [], "_D_": [], "_H_": [], "_L_": [], "_Preventative_": [], "_Conditions_": [], "_Diagnostic": [], "_Procedures_": [], "_Medications_": [], "_Recovery_":[]}

Survey_3_df = pd.DataFrame(Survey_3)
column_headers = list(Survey_3_df)

for column in column_headers:
    for i in range(len(Question_Type_Eval)):
        question = Question_Type_Eval[i]
        ques_title = Question_Names_Eval[i]
        if question in column:
            S3_Lists[ques_title].append(column)       
        
    for source in Response_Source:
        if source in column:
            S3_Lists[source].append(column)
    
    for domain in Medical_Domain:
        if domain in column:
            S3_Lists[domain].append(column)


dataset_3 = pd.DataFrame()
dataset_3.insert(0, "Question Type", 0)
dataset_3.insert(1, "Response Scores", 0)
dataset_3.insert(2, "Response Source", 0)
dataset_3.insert(3, "Random Header", 0)
dataset_3.insert(4, "Participant ID", 0)
dataset_3.insert(5, "Question ID", 0)


In [11]:
def Indicated_Source_Likert(Question, Dataset, Stat_dataset, Survey_Str, Survey_Dict) :
    
    Add = [" - D", " - H", " - L"]
    Ques = Survey_Dict[Question]

    for i in range(3):
        name = Question + Add[i]
        curr_set = Survey_Dict[Response_Source[i]]
        Sample = sorted(list(set(Ques) & set(curr_set)))

        for col in Sample:
            Temp_column = Dataset.loc[:, col]
            Question_ID = col
            Participants = Dataset.loc[:, "prolific_id"]

            for ind in Dataset.index:
                resp = Temp_column[ind]
                Participant_ID = Participants[ind]
                if str(Random_Source[ind]) == "DOCTOR":
                    if 0 <= float(resp) <= 5:
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, float(resp), name, "Doctor", Participant_ID, Question_ID]
                    
                if str(Random_Source[ind]) == "ARTIFICIAL INTELLIGENCE (A.I.)":
                    if 0 <= float(resp) <= 5:
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, float(resp), name, "AI", Participant_ID, Question_ID]
                
                if str(Random_Source[ind]) == "DOCTOR ASSISTED BY ARTIFICIAL INTELLIGENCE (A.I.)":
                    if 0 <= float(resp) <= 5:
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, float(resp), name, "Doctor + AI", Participant_ID, Question_ID]

    return("Done")





def Yes_or_No_S3(Question, Dataset, Stat_dataset, Survey_Dict):
    
    Add = [" - D", " - H", " - L"]
    Ques = Survey_Dict[Question]

    for i in range(3):
        name = Question + Add[i]
        curr_set = Survey_Dict[Response_Source[i]]
        Sample = sorted(list(set(Ques) & set(curr_set)))

        for col in Sample:
            Temp_column = Dataset.loc[:, col]
            Question_ID = col
            Participants = Dataset.loc[:, "prolific_id"]

            for ind in Dataset.index:
                resp = Temp_column[ind]
                Participant_ID = Participants[ind]

                if str(Random_Source[ind]) == "DOCTOR":
                    if resp == "Yes":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 1, name, "Doctor", Participant_ID, Question_ID]

                    if resp == "No":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 0, name, "Doctor", Participant_ID, Question_ID]
                    

                if str(Random_Source[ind]) == "ARTIFICIAL INTELLIGENCE (A.I.)":
                    if resp == "Yes":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 1, name, "AI", Participant_ID, Question_ID]

                    if resp == "No":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 0, name, "AI", Participant_ID, Question_ID]
                    

                if str(Random_Source[ind]) == "DOCTOR ASSISTED BY ARTIFICIAL INTELLIGENCE (A.I.)":
                    if resp == "Yes":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 1, name, "Doctor + AI", Participant_ID, Question_ID]

                    if resp == "No":
                        Stat_dataset.loc[len(Stat_dataset)] = [Question, 0, name, "Doctor + AI", Participant_ID, Question_ID]

    return("Done")



In [12]:
for ques in Question_Names_Eval:
    if ques != "Valid":
        Indicated_Source_Likert(ques, Survey_3, dataset_3, "Survey 3", S3_Lists)
    else:
        Yes_or_No_S3(ques, Survey_3, dataset_3, S3_Lists)


### Dataset --> CSV

In [13]:
dataset_1.to_csv("/work/data/3) final_data/Survey 1.csv")
dataset_1.to_csv("/work/data/3) final_data/Survey 1.csv")
dataset_2_1.to_csv("/work/data/3) final_data/Survey 2 - 1.csv")
dataset_2_2.to_csv("/work/data/3) final_data/Survey 2 - 2.csv")
dataset_3.to_csv("/work/data/3) final_data/Survey 3.csv")

### Dataset Details

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28896330-bd0f-4f0f-a160-af87c463253f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>